In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# df_filtrado = df.loc[df['event']=='conversion',:]
grouped = df.groupby(['person', 'event']).agg({'timestamp':np.max})

#### Verifico que matchee alguna fecha para algun user

In [45]:
grouped.head()

timestamp
person   event                              
0008ed71 checkout        2018-05-17 16:28:37
         generic listing 2018-05-17 16:22:06
         visited site    2018-05-17 16:21:54
00091926 ad campaign hit 2018-05-31 19:52:03
         brand listing   2018-05-31 19:52:03

In [46]:
df.loc[df['person']=='00091926', ['timestamp']].sort_values(by='timestamp', ascending=False).head(1)

,timestamp
1777801,2018-05-31 19:52:03


#### Ahora lo pasamos a dias que pasaron desde su ultima conexión

In [47]:
grouped['days_elapsed'] = pd.to_datetime('2018-06-01') - grouped['timestamp']
grouped['days_elapsed'] = grouped['days_elapsed'] / np.timedelta64(1, 'D')

In [49]:
grouped.sort_values(by='days_elapsed', ascending=False).head(10)

timestamp  days_elapsed
person   event                                              
0f4e2a4b search engine hit 2018-01-01 08:45:29    150.635081
99841b84 generic listing   2018-01-01 20:30:58    150.145162
a66e8424 searched products 2018-01-01 22:57:02    150.043727
a0084db9 visited site      2018-01-01 23:56:17    150.002581
         brand listing     2018-01-01 23:56:18    150.002569
fb19ca5a searched products 2018-01-02 00:41:26    149.971227
9ea14cff viewed product    2018-01-02 01:34:25    149.934433
         visited site      2018-01-02 01:34:25    149.934433
         ad campaign hit   2018-01-02 01:34:25    149.934433
         search engine hit 2018-01-02 01:34:25    149.934433

#### Que se puede hacer con esto:

 + podemos tener el ultimo evento que realizó y hace cuantos días.
 + podemos tener la cantidad de días que pasaron desde la ultima conexión.
 + podemos tener la cantidad de días que pasaron desde que hizo X event.

###### Voy a hacer la cantidad de días que pasaron desde el último event del usuario:

In [50]:
days_from_last_event = grouped.groupby('person').agg({'days_elapsed':np.max})

In [51]:
days_from_last_event.head()

,days_elapsed
person,
0008ed71,14.318125
00091926,5.978623
00091a7a,66.381123
000ba417,14.513623
000c79fe,2.980706


In [52]:
days_from_last_event.to_csv('data/days_elapsed_from_last_event.csv')